In [339]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [340]:
root = os.getcwd()
parent = os.path.join(root, 'data', 'rain')
filename = os.listdir(parent)[0]
file_path = os.path.join(parent, filename)

In [341]:
## read file into a dataframe
df_orig = pd.read_csv(file_path)

In [342]:
## Create a copy of the original dataframe
df = df_orig.copy()

### Plan of action to pre-process the data

1. Target variable for this data set is the RainTomorrow variable
2. There are complications here since the data is time series data, there could e strong autocorrelations in the data
3. Also given that there are locations in the data set there could also be strong spatial correlations that we need to watch out for
4. Want to test the data for auto-correlation using: Durbin Watson test or Ljung-Box tets
    a. The Adj. Dickey-Fuller test (ADF) is used to check for stationarity in timeseries data
    b. ACF & PACF plot are also a visual way to check for Auto-correlation
5. Train test split. Given the data is imbalanced interms of the labels, sugegst to do a startified sampling bsed on label set
6. Need to draw out the pre-processing pipeline:
    a. Missing values ffill and bfill. 
        1. Try median imputation as well 
        2. Try 3-day moving average as imputation technique
        3. Check the overall statistics using describe and check difference between all three
    b. Add the derived variable: Temp range
    c. Scale (Standardize/Normalize) the numerical variable
    d. Check if any of the variables can be discretized
7. Have a bunch of candidate models for the task
8. Use cross validation for model selection and hyper-parameter tuning (Grid/Randomized search CV)
9. Evaluate Generalization error
10.Monitor the performance of the model
11. Check if updated data is available on the Australian website to download

In [343]:
## Getting a high level feel of the data set
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [344]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [345]:
## Start by conveting the Date column to datetime
df.Date = pd.to_datetime(df.Date, format='%Y-%m-%d')

## First step is check the target variable

### We may have an opportunity to fill missing values in target based on the values in RainToday column
pd.crosstab(df.RainToday.isna(), df.RainTomorrow.isna())

RainTomorrow,False,True
RainToday,,
False,140787,1412
True,1406,1855


In [346]:
# Cleaning up target variable

### So start by dropping columns with both values missing
df = df[~((df.RainTomorrow.isna())&(df.RainToday.isna()))]

## Check if we can impute the Rain tomorrow columns with Rain Today from next day

df['rainTom_prev_day'] = df.sort_values(by=['Location', 'Date']).groupby('Location')['RainTomorrow'].shift(1)
df['rainTod_next_day'] = df.sort_values(by=['Location', 'Date']).groupby('Location')['RainToday'].shift(-1)

# # quick check
# df[(df.Date.dt.year>=2009) & (df.Date <= df.Date.min()+pd.Timedelta(value=1, unit='days'))]   # works since NaNs are populated at the first day of each locaion


df['RainTomorrow'] = df['RainTomorrow'].combine_first(df['rainTod_next_day'])
df['RainToday'] = df['RainToday'].combine_first(df['rainTom_prev_day'])

# # Doesn't look like there is much opportunity here
df[df.RainTomorrow.isna()|df.RainToday.isna()]

## Dropping rows where values for target variable are missing
df = df[df.RainTomorrow.notna()].drop(['rainTod_next_day', 'rainTom_prev_day'], axis=1)

In [347]:
# Incomplete data.
## A. Dropping data prior to a particular timeperiod due to data being incomplete
df = df[df.Date.dt.year>=2009]

In [348]:
# Handling missing values
## Check for the current % of missing values in the dataset
def get_missing(df):
    s = df.isna().sum(axis=0)*100/len(df)
    s = s[s>0].to_frame(name='miss_pct')
    s = s.join(df.dtypes.to_frame(name='data_type'), how='left')
    # s = s.merge(df.dtypes.to_frame(),how='left', left_index=True, right_index=True)
    return s

#### Calculating % of missing values in each column
get_missing(df)

,miss_pct,data_type
MinTemp,0.448937,float64
MaxTemp,0.227328,float64
Rainfall,0.992951,float64
Evaporation,43.262371,float64
Sunshine,48.252148,float64
WindGustDir,6.350171,object
WindGustSpeed,6.307994,float64
WindDir9am,7.093633,object
WindDir3pm,2.687188,object
WindSpeed9am,0.943626,float64


In [349]:
# Handling missing values

## A. Dropping columns with high % of missing values
miss_cols = get_missing(df)
#### dropping coumns where there are more than 20% of values missing
miss_cols = miss_cols[miss_cols.miss_pct>20].index.values ## .values returs an nd-numpy array
miss_cols
df = df.drop(miss_cols, axis=1)

In [350]:
## B. Dropping Locations with missing values (i.e. data not being captured)

miss_loc = df.groupby('Location').count()
miss_loc = (miss_loc == 0).any(axis=1)
miss_loc = miss_loc[miss_loc==True].index.values

#### Check count of data pointsfor these locations in the dataset
df['Location'].value_counts()/len(df)  # Most locations are pretty uniformly distributed interm of count

#### Based on the above looks like dropping these locations is not going to significantly impact the analysis
#### Dropping the locations missing ceratain variables
df = df[~df.Location.isin(miss_loc)]    # given that this is not an index based drop, instead of using the drop function we simply filter the dataset

In [351]:
df[df.Rainfall.notna()&df.RainToday.isna()][['Rainfall', 'RainToday']]
# df.Rainfall

,Rainfall,RainToday


In [352]:
# Imputation of Missing values

## Opportunity to impute Temp3PM with MaxTemp and vice-versa
df['Temp3pm'] = df['Temp3pm'].combine_first(df['MaxTemp']) 
df['MaxTemp'] = df['MaxTemp'].combine_first(df['Temp3pm']) 

## Rainfall. If Raintoday = No, then rainfall is zero 
df['Rainfall'] = np.where(df.RainToday == 'No', 0.0, df.Rainfall)
# df[df.Rainfall.notna()&df.RainToday.isna()][['Rainfall', 'RainToday']]

## Method 1: Forward & backward filling
# df2 = df.sort_values(by=['Location', 'Date']).groupby(by=['Location']).fillna(axis=0, method='ffill')\
#             .fillna(axis=0, method='bfill')
# df = df2.join(df['Location'], how='left')

In [353]:
# DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False
#                , right_index=False, sort=False, suffixes=('_x', '_y'), copy=None, indicator=False, validate=None)

In [354]:
## Imputation if missing values for categorical variables

cat_miss = get_missing(df)
cat_miss = cat_miss[cat_miss.data_type=='object'].index.values.tolist()

# ## Forward filling missing categorical variables

df[cat_miss] = df[['Location', 'Date']+cat_miss].sort_values(by=['Location', 'Date'])\
        .groupby('Location').fillna(method='ffill', axis=0).fillna(method='bfill', axis=0)[cat_miss]

In [355]:
# get_missing(df)
stats_orig = df.describe().T

In [356]:
## Imputation if missing values for numeric variables
## Three possible options
## Method 1: Imputation with ffill/bfill
## Method 2: Imputation with simple mean/median
## Method 3: Imputation with rolling mean/median (simple/exponential)

num_miss = get_missing(df)
num_miss = num_miss[num_miss.data_type!='object'].index.values.tolist()
num_miss = list(set(num_miss)-set(['Rainfall']))   # rainfall will be handled separately. Set needs a list passed set('Rainfall') didn't work 
num_miss

#### Method 1: Imputation with ffill/bfill
# df[num_miss] = df[['Location', 'Date']+num_miss].sort_values(by=['Location', 'Date'])\
#         .groupby('Location').fillna(method='ffill', axis=0).fillna(method='bfill', axis=0)[num_miss]
stats_1 = df.describe().T

#### Method 2a: Imputation with simple mean
# df[num_miss] = df[num_miss].combine_first(df.groupby('Location')[num_miss].transform('mean'))
stats_2a = df.describe().T

#### Method 2b: Imputation with simple median
# df[num_miss] = df[num_miss].combine_first(df.groupby('Location')[num_miss].transform('median'))
stats_2b = df.describe().T


In [400]:
df.groupby(['Location']).agg({'Date':['nunique', 'min', 'max']})
# import datetime as dt
# dt.datetime.strptime('2009-01-01', '%Y-%m-%d') - dt.datetime.strptime('2017-06-24', '%Y-%m-%d')

Date                      
                 nunique        min        max
Location                                      
Adelaide            2924 2009-01-01 2017-06-24
Albury              2981 2009-01-01 2017-06-25
AliceSprings        3000 2009-01-01 2017-06-24
BadgerysCreek       2928 2009-01-01 2017-06-25
Ballarat            2997 2009-01-01 2017-06-25
Bendigo             3003 2009-01-01 2017-06-25
Brisbane            2978 2009-01-01 2017-06-25
Cairns              2957 2009-01-01 2017-06-25
Canberra            2991 2009-01-01 2017-06-25
Cobar               2988 2009-01-01 2017-06-25
CoffsHarbour        2953 2009-01-01 2017-06-25
Dartmoor            2943 2009-03-05 2017-06-25
Darwin              3008 2009-01-01 2017-06-24
GoldCoast           2950 2009-01-01 2017-06-25
Hobart              3004 2009-01-01 2017-06-25
Katherine           1559 2013-03-01 2017-06-23
Launceston          2997 2009-01-01 2017-06-25
Melbourne           2251 2009-01-01 2017-06-25
MelbourneAirport    3009 2009-01-01 2017-06-25
Mildura             3007 2009-01-01 2017-06-25
Moree               2854 2009-01-01 2017-06-25
MountGambier        2999 2009-01-01 2017-06-24
Nhil                1569 2013-03-01 2017-06-25
NorahHead           2929 2009-01-01 2017-06-25
NorfolkIsland       2964 2009-01-01 2017-06-25
Nuriootpa           3002 2009-01-01 2017-06-24
PearceRAAF          2762 2009-01-01 2017-06-25
Perth               3009 2009-01-01 2017-06-25
PerthAirport        3009 2009-01-01 2017-06-25
Portland            2996 2009-01-01 2017-06-25
Richmond            2951 2009-01-01 2017-06-25
Sale                3000 2009-01-01 2017-06-25
Sydney              3002 2009-01-01 2017-06-25
SydneyAirport       3005 2009-01-01 2017-06-25
Townsville          3002 2009-01-01 2017-06-25
Tuggeranong         2967 2009-01-01 2017-06-25
Uluru               1521 2013-03-01 2017-06-24
WaggaWagga          2976 2009-01-01 2017-06-25
Walpole             2819 2009-01-01 2017-06-25
Watsonia            2999 2009-01-01 2017-06-25
Williamtown         2553 2009-01-01 2017-06-25
Witchcliffe         2952 2009-01-01 2017-06-25
Wollongong          2954 2009-01-01 2017-06-25
Woomera             2990 2009-01-01 2017-06-24

In [461]:
xyz = pd.date_range(start=df.Date.min(), end=df.Date.max(), freq='D').to_frame(name='Date')
xyz['key'] = 0
yzx = pd.DataFrame(df.loc[:,'Location'].unique(), columns=['Location'])
yzx['key'] = 0
xyz=xyz.merge(yzx, on='key', how='outer').drop('key', axis=1)
xyz = df.merge(xyz, on=['Location', 'Date'], how='outer')

xyz = xyz.groupby(['Location', 'Date'])['Date'].mean().diff().rename('gaps').reset_index()
xyz[xyz.gaps > pd.Timedelta(value=1, unit='days')]

# xyz

,Location,Date,gaps


In [429]:
## Check if time series has gaps. This I gotta fix at the beginning
# df.groupby(['Location', 'Date'])['Date'].mean().diff().rename('gaps').reset_index().groupby('gaps').agg({'Location':'nunique'
#                                                                                                          , 'Date':['min', 'max']})

d = df.groupby(['Location', 'Date'])['Date'].mean().diff().rename('gaps').reset_index()
miss_gt_32d_locs = d[d.gaps > pd.Timedelta(value=32, unit='days')].Location.unique()   # Directly drop these locations
d[d.gaps > pd.Timedelta(value=31, unit='days')]#.Location.unique()



# df.sort_values(by=['Location', 'date']).groupby(['Location'])[num_miss].rolling(3)

,Location,Date,gaps
1367,Adelaide,2013-01-01,32 days
4310,Albury,2013-01-01,32 days
7305,AliceSprings,2013-01-01,32 days
10261,BadgerysCreek,2013-01-01,32 days
13226,Ballarat,2013-01-01,32 days
16230,Bendigo,2013-01-01,32 days
19221,Brisbane,2013-01-01,32 days
22210,Cairns,2013-01-01,32 days
25158,Canberra,2013-01-01,32 days
28153,Cobar,2013-01-01,32 days


In [331]:
stats_orig.join(stats_1, rsuffix='_1')\
    .join(stats_2a, rsuffix='_2a')\
    .join(stats_2b, rsuffix='_2b')\
    .join(stats_3a, rsuffix='_3a')\
    .join(stats_3b, rsuffix='_3b')\
    
# stats_orig


,count,mean,std,min,25%,50%,75%,max,count_1,mean_1,std_1,min_1,25%_1,50%_1,75%_1,max_1
MinTemp,125212.0,12.388714,6.388666,-8.2,7.7,12.1,17.1,33.9,125212.0,12.388714,6.388666,-8.2,7.7,12.1,17.1,33.9
MaxTemp,125212.0,23.493390,7.009699,2.6,18.0,22.9,28.5,48.1,125212.0,23.493390,7.009699,2.6,18.0,22.9,28.5,48.1
Rainfall,124026.0,2.291718,8.592871,0.0,0.0,0.0,0.0,371.0,124026.0,2.291718,8.592871,0.0,0.0,0.0,0.0,371.0
WindGustSpeed,125212.0,40.097842,13.454369,6.0,31.0,39.0,48.0,135.0,125212.0,40.097842,13.454369,6.0,31.0,39.0,48.0,135.0
WindSpeed9am,125212.0,14.318843,8.788574,0.0,7.0,13.0,20.0,87.0,125212.0,14.318843,8.788574,0.0,7.0,13.0,20.0,87.0
WindSpeed3pm,125212.0,19.006405,8.665763,0.0,13.0,19.0,24.0,87.0,125212.0,19.006405,8.665763,0.0,13.0,19.0,24.0,87.0
Humidity9am,125212.0,68.438808,19.290425,0.0,57.0,69.0,83.0,100.0,125212.0,68.438808,19.290425,0.0,57.0,69.0,83.0,100.0
Humidity3pm,125212.0,50.893485,20.758560,0.0,36.0,52.0,65.0,100.0,125212.0,50.893485,20.758560,0.0,36.0,52.0,65.0,100.0
Pressure9am,125212.0,1017.637832,7.042950,980.5,1013.0,1017.6,1022.3,1041.0,125212.0,1017.637832,7.042950,980.5,1013.0,1017.6,1022.3,1041.0
Pressure3pm,125212.0,1015.259218,6.967964,977.1,1010.5,1015.1,1019.9,1039.6,125212.0,1015.259218,6.967964,977.1,1010.5,1015.1,1019.9,1039.6


In [244]:
# Adding additional columns to the dataset
df['temp_day_range'] = df['MaxTemp']-df['MinTemp']
df['3d_rolling_avg_Rainfall'] = df.sort_values(by=['Location', 'Date']).groupby(by=['Location'])[]

SyntaxError: invalid syntax (Temp/ipykernel_17188/4161569932.py, line 3)

In [ ]:
df.info()

In [ ]:
g = sns.relplot(data=df[df.Date.dt.year==2010]
            , x='Date', y='Rainfall'
            , col='RainTomorrow'#, hue='Location'
            , kind='line'
            , height=6, aspect=1
           )

In [ ]:
# sns.pairplot(data=df.sample(1000)
#              , diag_kind='kde'
#              , hue='RainToday'
#             )

In [ ]:
# alternatively: housing_num = housing.select_dtypes(include=[np.number])

### Plan of action to pre-process the data

1. Target variable for this data set is the RainTomorrow variable
2. There are complications here since the data is time series data, there could e strong autocorrelations in the data
3. Also given that there are locations in the data set there could also be strong spatial correlations that we need to watch out for
4. Want to test the data for auto-correlation using: Durbin Watson test or Ljung-Box tets
    a. The Adj. Dickey-Fuller test (ADF) is used to check for stationarity in timeseries data
    b. ACF & PACF plot are also a visual way to check for Auto-correlation
5. Train test split. Given the data is imbalanced interms of the labels, sugegst to do a startified sampling bsed on label set
6. Need to draw out the pre-processing pipeline:
    a. Missing values ffill and bfill. 
        1. Try median imputation as well 
        2. Try 3-day moving average as imputation technique
        3. Check the overall statistics using describe and check difference between all three
    b. Add the derived variable: Temp range
    c. Scale (Standardize/Normalize) the numerical variable
    d. Check if any of the variables can be discretized
7. Have a bunch of candidate models for the task
8. Use cross validation for model selection and hyper-parameter tuning (Grid/Randomized search CV)
9. Evaluate Generalization error
10.Monitor the performance of the model
11. Check if updated data is available on the Australian website to download


In [ ]:
import sklearn as sk